### 1. Download Shopify dataset and create table script:

#### Open terminal and run:
`gsutil cp gs://cs327e-open-access/spanner_shopify.zip .`

`unzip spanner_shopify.zip`

### 2. Configure Spanner CLI and create Shopify Database

In [2]:
%env GOPATH=/home/jupyter/go
%env PATH=$PATH:/usr/local/go/bin:$GOPATH/bin
%env SPANNER_EMULATOR_HOST=localhost:9010
%env CLI=/home/jupyter/go/bin/spanner-cli
%env SPAN_PROJECT=starlit-vim-303003
%env SPAN_INSTANCE=span-instance
%env SPAN_DATABASE=test-database

env: GOPATH=/home/jupyter/go
env: PATH=$PATH:/usr/local/go/bin:$GOPATH/bin
env: SPANNER_EMULATOR_HOST=localhost:9010
env: CLI=/home/jupyter/go/bin/spanner-cli
env: SPAN_PROJECT=starlit-vim-303003
env: SPAN_INSTANCE=span-instance
env: SPAN_DATABASE=test-database


In [3]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

Tables_in_test-database
T1


In [4]:
CLI_CONNECT="$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE"

In [5]:
!{CLI_CONNECT} -e "create database shopify"

In [6]:
%env SPAN_DATABASE=shopify

env: SPAN_DATABASE=shopify


In [7]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

In [8]:
!{CLI_CONNECT} -f /home/jupyter/spanner_shopify/create_tables.sql

In [9]:
!{CLI_CONNECT} -e "show tables"

Tables_in_shopify
key_benefits
apps
pricing_plans
apps_categories
pricing_plan_features
categories
reviews


In [10]:
!{CLI_CONNECT} -e "show create table key_benefits"

Table	Create Table
key_benefits	CREATE TABLE key_benefits (
  app_id STRING(50),
  title STRING(2000),
  description STRING(MAX),
) PRIMARY KEY(app_id, title)


In [11]:
!{CLI_CONNECT} -e "show create table apps"

Table	Create Table
apps	CREATE TABLE apps (
  id STRING(50),
  url STRING(1000),
  title STRING(2000),
  developer STRING(100),
  developer_link STRING(1000),
  icon STRING(1000),
  rating FLOAT64,
  reviews_count INT64,
) PRIMARY KEY(id)


In [12]:
!{CLI_CONNECT} -e "show create table pricing_plans"

Table	Create Table
pricing_plans	CREATE TABLE pricing_plans (
  id STRING(50),
  app_id STRING(50),
  title STRING(200),
  price FLOAT64,
) PRIMARY KEY(id)


In [13]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
) PRIMARY KEY(app_id, category_id)


In [14]:
!{CLI_CONNECT} -e "show create table pricing_plan_features"

Table	Create Table
pricing_plan_features	CREATE TABLE pricing_plan_features (
  pricing_plan_id STRING(50),
  app_id STRING(50),
  feature STRING(MAX),
) PRIMARY KEY(pricing_plan_id, app_id, feature)


In [15]:
!{CLI_CONNECT} -e "show create table categories"

Table	Create Table
categories	CREATE TABLE categories (
  id STRING(50),
  title STRING(2000),
) PRIMARY KEY(id)


In [16]:
!{CLI_CONNECT} -e "show create table reviews"

Table	Create Table
reviews	CREATE TABLE reviews (
  app_id STRING(50),
  author STRING(250),
  rating INT64,
  posted_at DATE,
) PRIMARY KEY(app_id, author)


### 3. Populate tables

In [17]:
from google.cloud import spanner
from google.cloud.spanner_v1 import param_types
import pandas as pd

In [18]:
span_instance="span-instance"
span_database="shopify"
spanner_client = spanner.Client()
instance = spanner_client.instance(span_instance)
database = instance.database(span_database)

In [19]:
df = pd.read_csv('/home/jupyter/spanner_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="apps",
        columns=("id", "url", "title", "developer", "developer_link", "icon", "rating", "reviews_count"),
        values=values
    )

In [20]:
df = pd.read_csv('/home/jupyter/spanner_shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="apps_categories",
        columns=("app_id", "category_id"),
        values=values
    )

In [21]:
df = pd.read_csv('/home/jupyter/spanner_shopify/categories.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="categories",
        columns=("id", "title"),
        values=values
    )

In [22]:
df = pd.read_csv('/home/jupyter/spanner_shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="key_benefits",
        columns=("app_id", "title", "description"),
        values=values
    )

In [23]:
df = pd.read_csv('/home/jupyter/spanner_shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="pricing_plans",
        columns=("id", "app_id", "title", "price"),
        values=values
    )

In [24]:
df = pd.read_csv('/home/jupyter/spanner_shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="pricing_plan_features",
        columns=("pricing_plan_id", "app_id", "feature"),
        values=values
    )

In [25]:
!{CLI_CONNECT} -e "select count(*) from apps"


3547


In [26]:
!{CLI_CONNECT} -e "select count(*) from apps_categories"


5383


In [27]:
!{CLI_CONNECT} -e "select count(*) from categories"


12


In [28]:
!{CLI_CONNECT} -e "select count(*) from key_benefits"


9541


In [29]:
!{CLI_CONNECT} -e "select count(*) from pricing_plans"


6275


In [30]:
!{CLI_CONNECT} -e "select count(*) from pricing_plan_features"


16270


In [31]:
def insert_app(transaction):

    row_ct = transaction.execute_update(
        "INSERT INTO apps (id, url, title, developer, developer_link, icon, reviews_count) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'https://apps.shopify.com/watchlist?surface_detail=inventory-management&surface_inter_position=1&surface_intra_position=9&surface_type=category'," 
          "'Back in Stock Product Alerts', "
          "'Swym Corporation', "
          "'https://apps.shopify.com/partners/developer-ca6a967f09890f68',"
          "'https://apps.shopifycdn.com/listing_images/9905a4c8f22cb4a3b0c32af55a58ec21/icon/e6d46a7e5e1df375d542d033aae80459.png?height=72&width=72',"
           "0)"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO apps_categories (app_id, category_id) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '737ad50051083aa051d127a53b3ac0da')"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    
    
    row_ct = transaction.execute_update(
        "INSERT INTO key_benefits (app_id, title, description) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'Drive Back In Stock Sales', 'Back in stock, out of stock, pre order & restock alerts')"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plans (id, app_id, title, price) "
        "VALUES ('961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', '683d06af-14c7-4733-9bde-ec5b699af996', "
                "'Free Trial', 14.99)"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    

    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plan_features (app_id, pricing_plan_id, feature) "
        " VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', "
                 "'Up to 250 alert requests/mo')"
    )
    
    print("{} record(s) inserted.".format(row_ct))
      
database.run_in_transaction(insert_app)

1 record(s) inserted.
1 record(s) inserted.
1 record(s) inserted.
1 record(s) inserted.
1 record(s) inserted.


In [32]:
def update_price(transaction):
    row_ct = transaction.execute_update(
            "UPDATE pricing_plans "
            "SET price = price / 2 "
            "WHERE price > 0"
        )

    print("{} record(s) updated.".format(row_ct))

database.run_in_transaction(update_price)

4780 record(s) updated.


In [33]:
def delete_app(transaction):
        row_ct = transaction.execute_update(
            "DELETE FROM apps WHERE id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )

        print("{} record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM apps_categories WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM key_benefits WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plans WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plan_features WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} record(s) deleted.".format(row_ct))

database.run_in_transaction(delete_app)

1 record(s) deleted.
1 record(s) deleted.
1 record(s) deleted.
1 record(s) deleted.
1 record(s) deleted.


In [34]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [35]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
  CONSTRAINT apps_cat_app_id_fk FOREIGN KEY(app_id) REFERENCES apps(id),
) PRIMARY KEY(app_id, category_id)


In [36]:
!{CLI_CONNECT} -e "CREATE INDEX app_rating_idx ON apps(rating)"

# Begin Project 3 Work 

In [37]:
# Populate reviews table
df = pd.read_csv('/home/jupyter/spanner_shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="reviews",
        columns=("app_id", "author", "rating", "posted_at"),
        values=values
    )

In [38]:
# Test to see if loaded
!{CLI_CONNECT} -e "select count(*) from reviews limit 5"


124601


In [39]:
# Describe table 
!{CLI_CONNECT} -e "show create table reviews"

Table	Create Table
reviews	CREATE TABLE reviews (
  app_id STRING(50),
  author STRING(250),
  rating INT64,
  posted_at DATE,
) PRIMARY KEY(app_id, author)


In [40]:
# Add Review
def insert_review(transaction):
    
    row_ct = transaction.execute_update(
        "INSERT INTO reviews (app_id, author, rating,posted_at) "
        " VALUES ('6286d4cd-3fcb-4ee3-bb3c-de051c28b83c', 'Jacob', 3,'2019-02-12')"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO reviews (app_id, author, rating,posted_at) "
        " VALUES ('6286d4cd-3fcb-4ee3-bb3c-de051c28b83c', 'Annie', 4,'2019-12-12')"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    

database.run_in_transaction(insert_review)

1 record(s) inserted.
1 record(s) inserted.


In [45]:
def update_price(transaction):
    row_ct = transaction.execute_update(
            "UPDATE apps "
            "SET reviews_count= 2 , rating=3 "
            "WHERE id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c'"
        )

    print("{} record(s) updated.".format(row_ct))

database.run_in_transaction(update_price)

1 record(s) updated.


Find and Add FK constraints

In [46]:
# apps
!{CLI_CONNECT} -e "show create table apps"

Table	Create Table
apps	CREATE TABLE apps (
  id STRING(50),
  url STRING(1000),
  title STRING(2000),
  developer STRING(100),
  developer_link STRING(1000),
  icon STRING(1000),
  rating FLOAT64,
  reviews_count INT64,
) PRIMARY KEY(id)


In [47]:
# apps_cat
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
  CONSTRAINT apps_cat_app_id_fk FOREIGN KEY(app_id) REFERENCES apps(id),
) PRIMARY KEY(app_id, category_id)


In [48]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories ADD FOREIGN KEY (app_id) REFERENCES apps(id)"

In [52]:
# categories
!{CLI_CONNECT} -e "show create table categories"

Table	Create Table
categories	CREATE TABLE categories (
  id STRING(50),
  title STRING(2000),
) PRIMARY KEY(id)


In [53]:
# key_benefits
!{CLI_CONNECT} -e "show create table key_benefits"

Table	Create Table
key_benefits	CREATE TABLE key_benefits (
  app_id STRING(50),
  title STRING(2000),
  description STRING(MAX),
) PRIMARY KEY(app_id, title)


In [54]:
# ADD FK
!{CLI_CONNECT} -e "ALTER TABLE key_benefits ADD FOREIGN KEY (app_id) REFERENCES apps(id)"

In [55]:
# pricing_plans
!{CLI_CONNECT} -e "show create table pricing_plans"

Table	Create Table
pricing_plans	CREATE TABLE pricing_plans (
  id STRING(50),
  app_id STRING(50),
  title STRING(200),
  price FLOAT64,
) PRIMARY KEY(id)


In [56]:
# ADD FK (app
!{CLI_CONNECT} -e "ALTER TABLE pricing_plans ADD FOREIGN KEY (app_id) REFERENCES apps(id)"

In [57]:
# pricing_plan_features
!{CLI_CONNECT} -e "show create table pricing_plan_features"

Table	Create Table
pricing_plan_features	CREATE TABLE pricing_plan_features (
  pricing_plan_id STRING(50),
  app_id STRING(50),
  feature STRING(MAX),
) PRIMARY KEY(pricing_plan_id, app_id, feature)


In [58]:
# ADD FK
!{CLI_CONNECT} -e "ALTER TABLE pricing_plan_features ADD FOREIGN KEY (app_id) REFERENCES apps(id)"
!{CLI_CONNECT} -e "ALTER TABLE pricing_plan_features ADD FOREIGN KEY (pricing_plan_id) REFERENCES pricing_plans(id)"

In [59]:
# reviews
!{CLI_CONNECT} -e "show create table reviews"

Table	Create Table
reviews	CREATE TABLE reviews (
  app_id STRING(50),
  author STRING(250),
  rating INT64,
  posted_at DATE,
) PRIMARY KEY(app_id, author)


In [60]:
# ADD FK
!{CLI_CONNECT} -e "ALTER TABLE reviews ADD FOREIGN KEY (app_id) REFERENCES apps(id)"

# Summary of Adding Fks
* Added FK to apps_categories.app_ID table pointing to app.id
* Added FK to apps_categories.catergory_id table pointing to catergory.id

* Added FK to reviews.app_ID table pointing to app.id

* Added FK to key_benefits.app_ID table pointing to app.id

* Added FK to pricing_plan.app_ID table pointing to app.id
* Added FK to pricing_plan.id table pointing to pricing_plan_features.prciing_plan_id

* Added FK to pricing_plan_features.app_id table pointing to apps.id

# Optimizing Code

In [ ]:
CREATE INDEX AlphaAuthors ON reviews(author ASC)

select * from apps a join reviews@{FORCE_INDEX=AlphaAuthors} r on a.id = r.app_id
where a.rating >= 5.0 and r.author = 'Funky Moose Records' ;